In [1]:
from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

from skimage.feature import hog
from skimage.transform import pyramid_gaussian
import joblib
from skimage import color
from imutils.object_detection import non_max_suppression
import imutils
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
from skimage.color import rgb2gray


Caltech_images_dir_test= 'dataset/test_hog/Images/'
#my_images_dir_test='testMyData/Images/'

Caltech_annotations_test ='dataset/test_hog/test.json'
#my_annotations_test='testMyData/myTest.json'

classes = ['person']
dataset = DataGenerator()
dataset.parse_json(images_dirs=[Caltech_images_dir_test],
                         annotations_filenames=[Caltech_annotations_test],
                         ground_truth_available=True,
                         include_classes='all',
                         verbose = True,
                         ret=False)

#img_height=650
#img_width=520
img_height=360
img_width=640

convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)
batch_size=1
generator = dataset.generate(batch_size=1,
                             shuffle=False,
                             transformations=[convert_to_3_channels,
                                              resize],
                             returns={'processed_images',
                                      'filenames',
                                      'inverse_transform',
                                      'original_images',
                                      'original_labels'},
                             keep_images_without_gt=False)

def sliding_window(image, stepSize,windowSize): 
    for y in range(0, image.shape[0],stepSize):  
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y: y + windowSize[1], x:x + windowSize[0]])

orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

model_path = 'model/modelSVM.npy'
model = joblib.load(model_path)



Processing 'test.json': 100%|████████████████████████████████████████████████████| 956/956 [00:00<00:00, 239645.87it/s]


In [2]:
import numpy as np
from sklearn.metrics import average_precision_score

iou_th = 0.5
thrsh = 4

TP = 0
all_ground = 0
all_det = 0
tru = np.array(1)
score = np.array(1)
for k in range(0, dataset.get_dataset_size()):

    batch_images, batch_filenames, batch_inverse_transforms, batch_original_images, batch_original_labels = next(
        generator)
    '''
    print("Image:", batch_filenames[0])
    print()
    print("Ground truth boxes:\n")
    print(np.array(batch_original_labels[0]))
    '''
    scale = 0
    detections = []
    img = cv2.imread(batch_filenames[0])
    # img_copo=cv2.imread(batch_filenames[0])
    img = rgb2gray(img)

    (winW, winH) = (64, 128)
    windowSize = (winW, winH)
    downscale = 1.5
    for resized in pyramid_gaussian(img, downscale=1.5):
        for (x, y, window) in sliding_window(resized, stepSize=10, windowSize=(winW, winH)):
            if window.shape[0] != winH or window.shape[1] != winW:
                continue
            fds = hog(window, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True)
            fds = fds.reshape(1, -1)
            pred = model.predict(fds)

            if pred == 1:
                if model.decision_function(fds) > thrsh:
                    '''
                    print("Detection:: Location -> ({}, {})".format(x, y))
                    print("Scale ->  {} | Confidence Score {} \n".format(scale, model.decision_function(fds)))
                    '''
                    detections.append(
                        (int(x * (downscale ** scale)), int(y * (downscale ** scale)), model.decision_function(fds),
                         int(windowSize[0] * (downscale ** scale)),  # create a list of all the predictions found
                         int(windowSize[1] * (downscale ** scale))))
        scale += 1

    clone = resized.copy()
    sc = [score[0] for (x, y, score, w, h) in detections]
    rects = np.array([[x, y, x + w, y + h] for (x, y, _, w, h) in detections])  # nms
    '''
    print("detection confidence score: ", sc)
    '''
    sc = np.array(sc)
    pick = non_max_suppression(rects, probs=sc, overlapThresh=0.3)
    first = True
    lp = 0
    for (xA, yA, xB, yB) in pick:
        if lp >= len(pick.shape):
            lp = len(pick.shape) - 1
        if first == True:
            first = False
            onlyOne = True
            box = np.array([xA, yA, xB, yB, pick.shape[lp]])
        else:
            box = np.vstack((box, [xA, yA, xB, yB, pick.shape[lp]]))
            onlyOne = False
        cv2.rectangle(img, (xA, yA), (xB, yB), (0, 0, 255), 2)
        lp = lp + 1

    # cv2.imshow('obr.', img)
    # cv2.waitKey()

    h = len(batch_original_labels[0])
    all_ground = all_ground + h

    if first == True:
        # zadna detekce ->
        all_det = all_det + 0
    else:
        if onlyOne == True:
            dk = 1
            all_det = all_det + dk
        else:
            dk = len(box)
            all_det = all_det + dk
        for l in range(0, dk):

            if onlyOne == True:
                xmin2 = box[0]
                ymin2 = box[1]
                xmax2 = box[2]
                ymax2 = box[3]

                coefi = box[4]

            else:
                xmin2 = box[l][0]
                ymin2 = box[l][1]
                xmax2 = box[l][2]
                ymax2 = box[l][3]

                coefi = box[l][4]

            Precision_decision = True
            z = 0

            for box1 in batch_original_labels[0]:
                xmin1 = box1[1]
                ymin1 = box1[2]
                xmax1 = box1[3]
                ymax1 = box1[4]

                xmin = max(xmin1, xmin2)
                ymin = max(ymin1, ymin2)
                xmax = min(xmax1, xmax2)
                ymax = min(ymax1, ymax2)

                interArea = abs(max((xmax - xmin, 0)) * max((ymax - ymin), 0))
                boxAArea = abs((xmax1 - xmin1) * (ymax1 - ymin1))
                boxBArea = abs((xmax2 - xmin2) * (ymax2 - ymin2))
                iou = interArea / float(boxAArea + boxBArea - interArea)

                #print(iou)

                if iou >= iou_th:
                    TP = TP + 1
                    Precision_decision = False
                    tru = np.append(tru, 1)
                    score = np.append(score, coefi)
                    break

            if Precision_decision == True:
                tru = np.append(tru, 0)
                score = np.append(score, coefi)

Precision=(TP / all_det)
Recall=(TP / all_ground)
Average=(average_precision_score(tru, score))
F1=2 * (Precision * Recall) / (Precision + Recall)


In [3]:
print('Práh modelu: ',thrsh)
print('Precision: ',Precision)
print('Recall: ',Recall)
print('AP: ',Average)
print('F1: ',F1)

Práh modelu:  4
Precision:  0.4706994328922495
Recall:  0.424190800681431
AP:  0.3994094013643653
F1:  0.44623655913978494
